# Geographical map for the places of posting twitter -- John(Litao) Zhou

This is a geographical analysis map used the folium library to analysis the places of posting tweets<br>
The heat map of time analysis is done to the sample No.50 and more sample would be done.<br>
reference: https://github.com/nancygaooo/Visualization-Practice/blob/master/Folium%20Visualization%20Examples.ipynb <br>
reference: https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-data<br>
reference(Mandarian):https://www.zhihu.com/question/33783546 

In [6]:
#import the required libriary used for the analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import json as js
import ast
from datetime import datetime

In [7]:
#open places file and handle the issue of single quotes
with open(r"places.txt", encoding="utf8") as corona_file:
    pl=[]
    line= corona_file.readlines()
    for row in range(len(line)):
        temp = ast.literal_eval(line[row])
        pl.append(temp)

    pls_df=pd.DataFrame(pl)

In [8]:
#helper function to calculate the average and apply them to all
def average(number1, number2):
    return (number1 + number2) / 2.0

#compute the average data of latitude and lontitude 
pls_df['Lontitude'] = pls_df.apply(lambda x: average (x['geo']['bbox'][0],x['geo']['bbox'][2]),axis = 1)
pls_df['Latitude'] = pls_df.apply(lambda x: average (x['geo']['bbox'][1],x['geo']['bbox'][3]),axis = 1)
pls_df

,full_name,place_type,id,country_code,country,geo,name,Lontitude,Latitude
0,"Düsseldorf, Allemagne",city,b7d3c12268abd20e,DE,Allemagne,"{'type': 'Feature', 'bbox': [6.688728, 51.1243...",Düsseldorf,6.814308,51.238447
1,"Manhattan, NY",city,01a9a39529b27f36,US,Etats-Unis,"{'type': 'Feature', 'bbox': [-74.026675, 40.68...",Manhattan,-73.968542,40.780709
2,"Connecticut, USA",admin,e86b380cfefcced5,US,Etats-Unis,"{'type': 'Feature', 'bbox': [-73.727776, 40.95...",Connecticut,-72.757385,41.500753
3,"नई दिल्ली, भारत",city,317fcc4b21a604d5,IN,Inde,"{'type': 'Feature', 'bbox': [76.84252, 28.3976...",नई दिल्ली,77.095086,28.638489
4,"Albuquerque, NM",city,813a485b26b8dae2,US,Etats-Unis,"{'type': 'Feature', 'bbox': [-106.7916912, 35....",Albuquerque,-106.632718,35.117003
...,...,...,...,...,...,...,...,...,...
118818,sri ramanujar temple,poi,07d9e8a713086002,IN,Inde,"{'type': 'Feature', 'bbox': [79.948974609375, ...",sri ramanujar temple,79.948975,12.968259
118819,somewhere in India,poi,0909a45fdf54c001,IN,Inde,"{'type': 'Feature', 'bbox': [77.846767, 29.674...",somewhere in India,77.846767,29.674320
118820,Muthara,poi,07d9cc9aef083003,KE,Kenya,"{'type': 'Feature', 'bbox': [37.79895620871095...",Muthara,37.798956,0.216096
118821,Pensnett Doctors,poi,07d9cb7e4d885000,GB,Royaume-Uni,"{'type': 'Feature', 'bbox': [-2.131352, 52.500...",Pensnett Doctors,-2.131352,52.500561


**Heat Map With Time**

In [ ]:
#open the data file and handle the issue of single quotes
with open(r"corona_tweets_50_data.txt", encoding="utf8") as corona_file:
    tw=[]
    line= corona_file.readlines()
    for row in range(len(line)):
        temp = ast.literal_eval(line[row])
        tw.append(temp)
    tws_df=pd.DataFrame(tw)

In [ ]:
#define helper function to clear the no places situation
def geo_nan(df):
    if isinstance(df.loc['geo'], float):
        return False
    if 'place_id' in df['geo']:
        return True
    return False

tws_df['geo_nan'] = tws_df.apply(geo_nan, axis=1)
tws_df = tws_df[tws_df['geo_nan'] == True]

In [ ]:
#merge the two dataframe with the geo-id
tws_df['geo_id'] = tws_df.apply(lambda x: x['geo']['place_id'],axis = 1)
tws_merge = pd.merge(tws_df,pls_df,how = 'inner',left_on = 'geo_id',right_on = 'id')

In [ ]:
#define the helper function to do the function take out the string of time and summarize them into the hour
def process_time(df):
    time_str = df['created_at'][11:19]
    time_whole = datetime.strptime(time_str, '%H:%M:%S')
    return int(datetime.strftime(time_whole, '%H'))

#make an empty column and apply the function
tws_merge['hour'] = np.NaN
tws_merge['hour'] = tws_merge.apply(process_time, axis=1)

In [ ]:
#initialize the map and put the data into the map
map_50_tweets = folium.Map(location=[43.651070, -79.347015],zoom_start = 13) 

from folium.plugins import HeatMapWithTime

# List comprehension to make out list of lists
heat_data = [[[row['Latitude'],row['Lontitude']] for index, row in tws_merge[tws_merge['hour'] == i].iterrows()] for i in range(0,24)]

# Plot it on the map
hm = HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)
hm.add_to(map_50_tweets)

#show the heat map
map_50_tweets


In [ ]:
#save the map
map_50_tweets.save('corono_50_tweets_time_heat_map.html')

**Heat Map for all locations**

In [ ]:
#define and initilize a world map
world_map = folium.Map(location=[43.651070, -79.347015], zoom_start=3)

from folium.plugins import HeatMap

#use the plugin for heat map and convert into the heat value
heatdata = pls_df[['Latitude','Lontitude']].values.tolist()

# add incidents to map
HeatMap(heatdata).add_to(world_map)

#show the heat map
world_map_heat


In [5]:
#save the map
world_map_heat.save('heat map for all location.html')

**Cluster Map for all locations**

In [ ]:
#define and initilize a world map
world_map_cluster = folium.Map(location=[43.651070, -79.347015], zoom_start=3)

#use the plugin for data gathering
from folium import plugins

# initialize a mark cluster object for the incidents in the dataframe
number_tweets = plugins.MarkerCluster().add_to(world_map_cluster)

# Loop through the twitter places and add each to the incidents feature group
for lat, lng, in zip(pls_df['Latitude'], pls_df['Lontitude']):
    number_tweets.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            color='#4966F7',
            fill=True,
            fill_color='grey',
            fill_opacity=0.7
        )
    )

#add the group to the map and show the map
world_map_cluster.add_child(number_tweets)


In [11]:
#save the map
world_map_cluster.save('cluster map for all location.html.html')